### 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd /content/drive/MyDrive/oss_drive

/content/drive/MyDrive/oss_drive


### imports 및 seed fix

In [ ]:
# 세션 만료시마다 실행필요
%pip install torchbearer
%pip install pkbar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
# 필요한 imports
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchbearer
from torchbearer import Trial
import pkbar
import numpy as np
import random
import json
import datetime
import configparser
# image file truncated error prevention
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# seed fix
SEED = 3
np.random.seed(SEED)
random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.manual_seed(SEED)

### 데이터 로더 생성 클래스

In [ ]:
class PyTorchData():
    def __init__(self, _dataType, config):
        '''
        input_dim : image size
        data_path : training data path
        batch_size : batch size
        '''

        if _dataType == "data":
            self.m_DataDim = int(config['input_dim'])
        elif _dataType == "image":
            self.m_ImageDim = int(config['input_dim'])
            self.m_DataPath = config['data_path']
            self.m_BatchSize = int(config['batch_size'])


    # training할 이미지 로더 생성 및 리턴
    def ImageTrain(self):
        transDatagen = transforms.Compose([transforms.Resize((self.m_ImageDim, self.m_ImageDim)),
                                           transforms.ToTensor()])

        trainPath = self.m_DataPath + '/training'
        trainFolder = torchvision.datasets.ImageFolder(root = trainPath,
                                                       transform = transDatagen) # 폴더 정리가 같은 클래스 별로 이미 데이터 정리되어있음

        trainLoader = DataLoader(trainFolder,
                                batch_size = self.m_BatchSize,
                                shuffle = True)

        print("Train Class [", trainLoader.dataset.class_to_idx, "]")
        print("Train Numbers [", len(trainLoader.dataset.imgs), "]")
        print("Train Batch Size [", trainLoader.batch_size, "]")

        return trainLoader

    # validation할 이미지 로더 생성 및 리턴
    def ImageValidation(self):
        transDatagen = transforms.Compose([transforms.Resize((self.m_ImageDim, self.m_ImageDim)),
                                           transforms.ToTensor()])

        validationPath = self.m_DataPath + '/validation'
        validationSet = torchvision.datasets.ImageFolder(root = validationPath,
                                                         transform = transDatagen) # 폴더 정리가 같은 클래스 별로 이미 데이터 정리되어있음

        validationLoader = DataLoader(validationSet,
                                      batch_size = self.m_BatchSize,
                                      shuffle = False)

        print("Validation Class [", validationLoader.dataset.class_to_idx, "]")
        print("Validation Numbers [", len(validationLoader.dataset.imgs),"]")
        print("Validation Batch Size [", validationLoader.batch_size,"]")

        return validationLoader

    # testing할 이미지 로더 생성 및 리턴
    def ImageTest(self):
        transDatagen = transforms.Compose([transforms.Resize((self.m_ImageDim, self.m_ImageDim)),
                                           transforms.ToTensor()])

        testDirectory = self.m_DataPath + '/testing'
        testSet = torchvision.datasets.ImageFolder(root = testDirectory,
                                                   transform = transDatagen) # 폴더 정리가 같은 클래스 별로 이미 데이터 정리되어있음

        testLoader = DataLoader(testSet,
                                batch_size = self.m_BatchSize,
                                shuffle = False)

        print("Test Class [", testLoader.dataset.class_to_idx, "]")
        print("Test Numbers [", len(testLoader.dataset.imgs), "]")
        print("Test Batch Size [", testLoader.batch_size,"]")

        return testLoader


### 모델 정의

In [ ]:
# 모델 구조 정의
class PillModel(nn.Module):

    # bulid cnn model
    def __init__(self, config):
        super(PillModel, self).__init__()
        '''
        ClassNum : class number
        '''
        self.m_ClassNum = int(config['class_num'])

        channel1 = 16
        channel2 = 32
        channel3 = 64
        conv1Size = 3
        conv2Size = 3
        poolSize = 2

        self.m_Conv1 = nn.Conv2d(in_channels = 3, out_channels = channel1, kernel_size = conv1Size, padding = 1)
        self.m_Pool1 = nn.MaxPool2d(poolSize, poolSize)
        self.m_Conv2 = nn.Conv2d(in_channels = channel1, out_channels = channel2, kernel_size = conv2Size, padding = 1)
        self.m_Pool2 = nn.MaxPool2d(poolSize, poolSize)
        self.m_Conv3 = nn.Conv2d(in_channels = channel2, out_channels = channel3, kernel_size = conv2Size, padding = 1)
        self.m_Pool3 = nn.MaxPool2d(poolSize, poolSize)

        self.m_Linear4 = nn.Linear(40000, 256)
        self.m_Drop4 = nn.Dropout2d(0.5)

        self.m_Linear5 = nn.Linear(256, self.m_ClassNum)
        self.m_Relu = nn.ReLU()

    # forward 연산 정의
    def forward(self, x):
        x = self.m_Relu(self.m_Conv1(x))
        x = self.m_Pool1(x)

        x = self.m_Relu(self.m_Conv2(x))
        x = self.m_Pool2(x)

        x = self.m_Relu(self.m_Conv3(x))
        x = self.m_Pool3(x)

        x = x.view(x.shape[0],-1)
        x = self.m_Relu(self.m_Linear4(x))
        x = self.m_Drop4(x)

        x = self.m_Linear5(x)
        return x

### 모델 만들기
모델 저장, 학습, 테스팅 작업 정의

In [ ]:
# 모델 hyperparam설정, 저장, 학습, 테스팅 주요 기능
class MakeModel():

    def __init__(self,config):
        '''
        learning_rate : learning rate
        epochs : epoch
        save_path : save path
        model_name : model save name
        '''
        self._epoch = int(config['epochs'])
        self._lr = float(config['learning_rate'])
        self._savePath = config['save_path']
        self._modelName = config['model_name']

    # 모델.pt 저장
    def SaveModel(self, _model, optimizer, _trainData, trainLoss):
        nowdate = datetime.datetime.now().strftime('%y%m%d_%H')
        ret = 0
        try:
            torch.save({'model_state_dict': _model.state_dict(),
                        'epoch': self._epoch,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': trainLoss,
                        'label_name':_trainData.dataset.classes},
                        self._savePath + '/' + self._modelName + '_PyTorchModel.pt')
            print("model saved [", self._savePath + '/' + self._modelName + "_PyTorchModel.pt ]")

        except PermissionError:
                torch.save(_model, './' + nowdate + '_' + self._modelName + '_PyTorchModel.pt')
                print('model saved [ ./' + nowdate + '_' + self._modelName + '_PyTorchModel.pt ]')

        except IOError as e:
            print("IOError except: ", e.errno)
            ret = 1

        return ret

    # 모델 학습
    def Training(self, _device, _model, _trainData, _valData):
        _model.train()

        optimizer = optim.Adam(_model.parameters(), lr = self._lr)
        criterion = torch.nn.CrossEntropyLoss()
        bestValLoss = float('inf')

        for epoch in range(self._epoch):
            trainLoss = 0.0
            trainSize = 0.0
            trainCorrect = 0.0

            print("Epoch {}/{}".format(epoch + 1, self._epoch))
            progress = pkbar.Kbar(target=len(_trainData), width = 25)

            # train
            for batchIdx, data in enumerate(_trainData):
                images, labels = data
                images, labels = images.to(_device), labels.to(_device)

                optimizer.zero_grad()
                outputs = _model(images)

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                trainLoss = loss.item()

                _, predicted = outputs.max(1)
                trainSize += labels.shape[0]
                trainCorrect += predicted.eq(labels.view_as(predicted)).sum().item()
                trainAccuracy = 100 * trainCorrect / trainSize

                progress.update(batchIdx, values = [("loss: ", trainLoss), ("acc: ", trainAccuracy)])

                del loss
                del outputs

            # validation
            with torch.no_grad():
                valLoss = 0.0
                valSize = 0.0
                valCorrect = 0.0

                for batchIdx, data in enumerate(_valData):
                    images, labels = data
                    images, labels = images.to(_device), labels.to(_device)

                    outputs = _model(images)
                    valLoss = criterion(outputs, labels).item()

                    _, predicted = outputs.max(1)
                    valSize += labels.shape[0]

                    valCorrect += predicted.eq(labels.view_as(predicted)).sum().item()
                    valAccuracy = 100 * valCorrect / valSize

                progress.add(1, values=[("val loss", valLoss), ("val acc", valAccuracy)])

            # if best loss value, save model
            if valLoss < bestValLoss:
                bestValLoss = valLoss
                ret = self.SaveModel(_model, optimizer, _trainData, trainLoss)

        return ret


    # 테스트 데이터로 모델 테스팅 (한 번만 진행되어야 함)
    def Testing(self, _device, _model, _testData):
        _model.eval()
        criterion = torch.nn.CrossEntropyLoss()

        testLoss = 0.0
        testSize = 0.0
        testCorrect = 0.0

        progress = pkbar.Kbar(target=len(_testData), width = 25)

        with torch.no_grad():
            for batchIdx, data in enumerate(_testData):
                images, labels = data
                images, labels = images.to(_device), labels.to(_device)
                outputs = _model(images)

                testLoss = criterion(outputs, labels).item()

                _, predicted = outputs.data.max(1)
                testSize += labels.shape[0]
                testCorrect += predicted.eq(labels.view_as(predicted)).sum().item()
                accuracy = 100 * testCorrect / testSize

                progress.update(batchIdx, values = [("test loss: ", testLoss), ("test acc: ", accuracy)])

            testLoss /= len(_testData.dataset)
        progress.add(1)



### main

In [ ]:
# 해당 파일 실행 시 실행되는 main
class PyTorchMain():

    def __init__(self):
        config = configparser.ConfigParser()
        config.read('./Modelings/modeling_config.ini', encoding='UTF-8')

        self.m_Device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.m_cPytorchModel = PillModel(config['PT_model_info'])

        self.m_cPytorchData = PyTorchData("image", config['PT_model_info'])

        self.m_cMakeModel = MakeModel(config['PT_model_info'])


    def main(self):
        print("\n[ Model ]\n")
        model = self.m_cPytorchModel.to(self.m_Device)
        print(model)

        # load dataset
        print("\n[ Data ]\n")
        trainData = self.m_cPytorchData.ImageTrain()
        valData = self.m_cPytorchData.ImageValidation()
        testData = self.m_cPytorchData.ImageTest()

        # training
        print("\n[ Training ]\n")
        ret = self.m_cMakeModel.Training(_device = self.m_Device,
                                         _model = model,
                                         _trainData = trainData,
                                         _valData = valData)
        if ret == 0 or ret == 1:
            # testing
            print("\n[ Testing ]\n")
            self.m_cMakeModel.Testing(_device = self.m_Device,
                                      _model = model,
                                      _testData = testData)

        '''
        class_to_idx json 파일로 출력.
        f = open("C:\oss_medi\WhatIsMethIs-Model\model\Modelings\model\trainData_class_to_idx_230828.json", 'w')
        f.write(json.dumps(trainData.dataset.class_to_idx))
        f.close()
        '''

In [ ]:
# 해당 파일이 모듈로서 말고 직접 실행될 때
# ex) python ./Modelings/Modeling.py [config파일명 있어도 되고 없어도됨]
if __name__ == '__main__':
    print("Modeling.py 실행시작")
    mainClass = PyTorchMain()
    mainClass.main()
    print('####### Modeling.py 실행 finish #######')

Modeling.py 실행시작

[ Model ]

PillModel(
  (m_Conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (m_Pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (m_Conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (m_Pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (m_Conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (m_Pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (m_Linear4): Linear(in_features=40000, out_features=256, bias=True)
  (m_Drop4): Dropout2d(p=0.5, inplace=False)
  (m_Linear5): Linear(in_features=256, out_features=200, bias=True)
  (m_Relu): ReLU()
)

[ Data ]

Train Class [ {'29002': 0, '34342': 1, '37990': 2, '39916': 3, '40122': 4, '40720': 5, '40767': 6, '40792': 7, '40837': 8, '40949': 9, '40953': 10, '40990': 11, '40991': 12, '41097': 13, '41107': 14, '41169': 15, '41170': 16, '41172': 17, '41207

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


141/141 [=========================] - 2505s 18s/step - loss: : 2.4920 - acc: : 15.2130 - val loss: 1.7872 - val acc: 57.7922
model saved [ ./Modelings/model/230828_model_01_PyTorchModel.pt ]
Epoch 2/10
141/141 [=========================] - 55s 392ms/step - loss: : 0.7952 - acc: : 65.7255 - val loss: 0.5197 - val acc: 77.4351
model saved [ ./Modelings/model/230828_model_01_PyTorchModel.pt ]
Epoch 3/10
141/141 [=========================] - 55s 392ms/step - loss: : 0.4269 - acc: : 82.9164 - val loss: 0.7432 - val acc: 81.9805
Epoch 4/10
141/141 [=========================] - 55s 388ms/step - loss: : 0.2965 - acc: : 87.4791 - val loss: 0.0759 - val acc: 92.2078
model saved [ ./Modelings/model/230828_model_01_PyTorchModel.pt ]
Epoch 5/10
141/141 [=========================] - 56s 398ms/step - loss: : 0.2089 - acc: : 92.0262 - val loss: 0.2154 - val acc: 91.7208
Epoch 6/10
141/141 [=========================] - 55s 388ms/step - loss: : 0.1325 - acc: : 95.4101 - val loss: 0.0417 - val acc: 94.80